In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimage
import cv2
import numpy as np


In [2]:
import glob
vehicle_images=glob.glob("dataset/vehicles/*/*.png")
non_vehicle_images=glob.glob("dataset/non-vehicles/*/*.png")
print("Dataset Contains: {} Vehicle and {} Non-Vehicle images".format(len(vehicle_images),len(non_vehicle_images)))

Dataset Contains: 8792 Vehicle and 8968 Non-Vehicle images


In [3]:
from skimage.feature import hog
def bin_spatial(img, size=(16, 16)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

def color_hist(img, nbins=16, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

def get_features(img,cell_per_block=2,pix_per_cell=16,orient=11, color_hist_bins=16, img_resize=(64,64)):
    if(img.shape[0]!=img_resize[0]):
        img=cv2.resize(img,img_resize)
#     returns the feature vector for the image
    yuv_image=cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    feat1=hog(yuv_image[:,:,0], orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell), cells_per_block=(cell_per_block, cell_per_block), visualise=False, feature_vector=True)
    feat2=hog(yuv_image[:,:,0], orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell), cells_per_block=(cell_per_block, cell_per_block), visualise=False, feature_vector=True)
    feat3=hog(yuv_image[:,:,0], orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell), cells_per_block=(cell_per_block, cell_per_block), visualise=False, feature_vector=True)
#    get spacial features
    return np.concatenate((feat1,feat2,feat3,bin_spatial(img),color_hist(img)))

Read the images and create X_raw, y_raw


In [4]:
vehicle_features=[]
for vehicle_image in vehicle_images:
    vehicle_features.append(get_features(cv2.cvtColor(cv2.imread(vehicle_image),cv2.COLOR_BGR2RGB)))
non_vehicle_features=[]
for non_vehicle_image in non_vehicle_images:
    non_vehicle_features.append(get_features(cv2.cvtColor(cv2.imread(non_vehicle_image),cv2.COLOR_BGR2RGB)))


In [5]:
print("Vehicle Features: {} ".format(len(vehicle_features)))
print("Shape: {}".format(vehicle_features[0].shape))

Vehicle Features: 8792 
Shape: (2004,)


Scale the X features

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

X_raw = np.vstack((vehicle_features,non_vehicle_features)).astype(np.float64)
y_raw = np.hstack((np.ones(len(vehicle_features)),np.zeros(len(non_vehicle_features))))

stdscaler = StandardScaler().fit(X_raw)
X_raw_scaled = stdscaler.transform(X_raw)

# Test train split
X_train, X_test, y_train, y_test = train_test_split(
    X_raw_scaled, y_raw, test_size=0.2, random_state=12)

/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# Size of dataset
print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))

X_train shape: (14208, 2004)
X_test shape: (3552, 2004)
y_train shape: (14208,)
y_test shape: (3552,)


SVM GridSearch CV

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

linsvm = LinearSVC(C=0.01)
linsvm.fit(X_train,y_train)



LinearSVC(C=0.01, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

Test SVM calassifier score on X_test

In [10]:
from sklearn.metrics import accuracy_score

y_pred=linsvm.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.979448198198


Save the model to re-use (model persistance)

In [11]:
from sklearn.externals import joblib

joblib.dump(linsvm,'model_svm4.pkl')
joblib.dump(stdscaler, 'stdscaler4.pkl')
# Save the data
joblib.dump(np.array([X_train, X_test, y_train, y_test]),'dataset4_bkp.pkl')

['dataset4_bkp.pkl']